# **Object Detection - Models Training**

---

**Mateus Aleixo - 124256**

---

In [1]:
from collections import Counter
from glob import glob
import os
import random
import shutil
import torch
from ultralytics import YOLO

In [2]:
def get_image_label_pairs(
    image_root, annotation_root, image_ext="jpg", label_ext="txt"
):
    pairs = []
    image_files = glob(os.path.join(image_root, "**", f"*.{image_ext}"), recursive=True)

    for img in image_files:
        rel_path = os.path.relpath(img, image_root)
        label = os.path.join(
            annotation_root, os.path.splitext(rel_path)[0] + f".{label_ext}"
        )

        if os.path.exists(label):
            pairs.append((img, label))
        else:
            print(f"Warning: Label file not found for image {img}")

    return pairs

In [3]:
image_root = "../data/augmented_data/frames"
annotation_root = "../data/annotations"
pairs = get_image_label_pairs(image_root, annotation_root)
print(f"Found {len(pairs)} image-label pairs.")

Found 3864 image-label pairs.


In [4]:
empty_files = []

for root, _, files in os.walk(annotation_root):
    for file in files:
        file_path = os.path.join(root, file)

        if os.stat(file_path).st_size == 0:
            empty_files.append(file_path)

print(f"Found {len(empty_files)} empty label files.")

Found 0 empty label files.


In [5]:
label_counts = Counter()

for root, _, files in os.walk(annotation_root):
    for file in files:
        with open(os.path.join(root, file), "r") as f:
            lines = f.readlines()

            for line in lines:
                class_id = line.split()[0]
                label_counts[class_id] += 1

label_counts

Counter({'1': 7728, '2': 7728, '3': 7728, '0': 7726, '4': 7694})

In [6]:
# Paths
image_dir = "../data/augmented_data/frames"
label_dir = "../data/annotations"
output_dir = "../data/split_data"

# Create output folders
for split in ["train", "val"]:
    os.makedirs(os.path.join(output_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(output_dir, "labels", split), exist_ok=True)

# Get all images
all_images = []
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.endswith(".jpg") or file.endswith(".png"):
            all_images.append(os.path.join(root, file))

# Shuffle and split
random.shuffle(all_images)
train_ratio = 0.8
train_count = int(len(all_images) * train_ratio)

train_images = all_images[:train_count]
val_images = all_images[train_count:]


# Move files
def move_files(image_list, split):
    for img_path in image_list:
        # Get label path
        rel_path = os.path.relpath(img_path, image_dir)
        label_path = os.path.join(label_dir, os.path.splitext(rel_path)[0] + ".txt")

        # Define new paths
        new_img_path = os.path.join(
            output_dir, "images", split, os.path.basename(img_path)
        )
        new_label_path = os.path.join(
            output_dir, "labels", split, os.path.basename(label_path)
        )

        # Copy files
        shutil.copy(img_path, new_img_path)
        if os.path.exists(label_path):
            shutil.copy(label_path, new_label_path)


# Move train & val files
move_files(train_images, "train")
move_files(val_images, "val")

print(f"Dataset split completed: {len(train_images)} train, {len(val_images)} val")

Dataset split completed: 3091 train, 773 val


In [7]:
model = YOLO(verbose=False)

results = model.train(
    data="../data/dataset.yaml",
    epochs=50,
    batch=8,
    imgsz=416,
    device="cuda" if torch.cuda.is_available() else "cpu",
    optimizer="AdamW",
    lr0=0.001,
    weight_decay=0.0005,
)

New https://pypi.org/project/ultralytics/8.3.101 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.40  Python-3.12.9 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=../data/dataset.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=416, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_f

train: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\train... 2047 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2047/2047 [00:03<00:00, 578.66it/s]


train: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


c:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\.venv\Lib\site-packages\ultralytics\data\augment.py:1850: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning C:\Users\mateu\Downloads\UA\Cadeiras\Seminário\Object_Detection\data\split_data\labels\val... 711 images, 0 backgrounds, 0 corrupt: 100%|██████████| 711/711 [00:00<00:00, 754.07it/s]

val: New cache created: C:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\data\split_data\labels\val.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      50/50         0G     0.6781      0.352     0.7889         70        416: 100%|██████████| 256/256 [02:31<00:00,  1.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:18<00:00,  2.44it/s]


                   all        711       7106      0.993      0.994      0.992      0.855

50 epochs completed in 2.396 hours.
Optimizer stripped from c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\last.pt, 5.5MB
Optimizer stripped from c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\best.pt, 5.5MB

Validating c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train\weights\best.pt...
Ultralytics 8.3.40  Python-3.12.9 torch-2.6.0+cpu CPU (AMD Ryzen 7 6800H with Radeon Graphics)
YOLO11n summary (fused): 238 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 45/45 [00:16<00:00,  2.80it/s]


                   all        711       7106      0.993      0.994      0.992      0.854
                  blue        711       1422      0.995      0.996      0.992      0.869
                 green        711       1422      0.992      0.987      0.992      0.837
                orange        711       1422      0.995      0.996      0.994      0.861
                   red        711       1422      0.994      0.996      0.993      0.858
                yellow        709       1418       0.99      0.993      0.992      0.847
Speed: 0.4ms preprocess, 16.1ms inference, 0.0ms loss, 0.2ms postprocess per image
Results saved to c:\Users\mateu\Downloads\UA\Cadeiras\Seminrio\Object_Detection\ultralytics\runs\detect\train
